In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 218.4MB 62kB/s 
     |████████████████████████████████| 204kB 45.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814406 sha256=f0ac8e24e0938f833baa90c597f7fafaa14b9e90e34319b1086a8f1ea66fcd81
  Stored in directory: /root/.cache/pip/wheels/1e/5e/6a/17e906c94ec7246f260330a66e44a06a0809033ba2738a74a8
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## 2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de                  producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,                descripción del producto, categoría). Se pide resolver utilizando PySpark: A. Cuál es el producto más vendido. B. Cuál es la categoría de productos más vendida. C. Cuál es el top5 de productos más vendidos generando un RDD con (código de              producto, descripción, cantidad de ventas) D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando               para este análisis solo los productos que cuenten con al menos 50 ventas en el último                año. E. Idem anterior, pero calculando la categoría de productos que registró mayor variación            de precios en el último año. 

In [0]:
ventas = [('2019/02/28',1016,101.70),
('2019/11/24',1052,117.15),
('2019/11/02',1028,121.34),
('2019/06/29',1058,110.51),
('2019/01/04',1063,106.46),
('2019/04/30',1081,106.08),
('2019/05/26',1028,109.38),
('2019/03/10',1056,112.57),
('2019/05/01',1063,111.13),
('2019/10/11',1075,108.76),
('2019/02/28',1032,109.37),
('2019/09/30',1099,102.03),
('2019/04/25',1004,104.34),
('2019/12/29',1019,125.68),
('2019/06/07',1079,138.58),
('2019/10/10',1035,104.19),
('2019/12/03',1036,121.15),
('2019/04/16',1007,132.15),
('2019/07/06',1027,116.56),
('2019/05/08',1060,130.58),
('2019/02/15',1061,129.28),
('2019/06/10',1079,111.61),
('2019/09/28',1011,112.80),
('2019/01/10',1092,121.92),
('2019/10/09',1021,111.35),
('2019/03/03',1034,125.00),
('2019/10/17',1042,111.82),
('2019/10/01',1065,115.62),
('2019/11/09',1098,139.23),
('2019/08/22',1074,131.38),
('2019/01/10',1042,119.79),
('2019/11/30',1025,101.80),
('2019/08/08',1061,130.64),
('2020/01/01',1046,123.01),
('2019/10/16',1059,123.48),
('2019/01/02',1040,137.61),
('2019/12/23',1042,115.74),
('2019/09/01',1066,133.01),
('2019/03/07',1073,101.24),
('2019/01/28',1094,130.48),
('2019/01/28',1018,131.63),
('2019/06/28',1056,110.77),
('2019/02/05',1035,104.11),
('2019/01/13',1043,126.91),
('2019/07/22',1030,119.15),
('2019/06/15',1035,139.01),
('2019/12/26',1040,139.21),
('2019/03/13',1053,123.76),
('2019/07/31',1065,127.39),
('2019/10/09',1081,123.06)]

productos = [(1004,'NaN','alimento'),
(1007,'NaN','alimento'),
(1011,'NaN','alimento'),
(1016,'NaN','alimento'),
(1018,'NaN','alimento'),
(1019,'NaN','alimento'),
(1021,'NaN','alimento'),
(1025,'NaN','entretenimiento'),
(1027,'NaN','entretenimiento'),
(1028,'NaN','entretenimiento'),
(1030,'NaN','entretenimiento'),
(1032,'NaN','entretenimiento'),
(1034,'NaN','entretenimiento'),
(1035,'NaN','entretenimiento'),
(1035,'NaN','entretenimiento'),
(1036,'NaN','bazar'),
(1040,'NaN','bazar'),
(1042,'NaN','bazar'),
(1043,'NaN','bazar'),
(1046,'NaN','bazar'),
(1052,'NaN','indumentaria'),
(1053,'NaN','indumentaria'),
(1056,'NaN','indumentaria'),
(1058,'NaN','indumentaria'),
(1059,'NaN','indumentaria'),
(1060,'NaN','indumentaria'),
(1061,'NaN','indumentaria'),
(1063,'NaN','indumentaria'),
(1065,'NaN','indumentaria'),
(1066,'NaN','indumentaria'),
(1073,'NaN','perfumeria'),
(1074,'NaN','perfumeria'),
(1075,'NaN','perfumeria'),
(1079,'NaN','perfumeria'),
(1081,'NaN','cuidadoCorporal'),
(1092,'NaN','cuidadoCorporal'),
(1094,'NaN','cuidadoCorporal'),
(1098,'NaN','cuidadoCorporal'),
(1099,'NaN','cuidadoCorporal')]

In [0]:
productosRDD = sc.parallelize(productos)
ventasRDD = sc.parallelize(ventas)

In [6]:
productosRDD.collect()

[(1004, 'NaN', 'alimento'),
 (1007, 'NaN', 'alimento'),
 (1011, 'NaN', 'alimento'),
 (1016, 'NaN', 'alimento'),
 (1018, 'NaN', 'alimento'),
 (1019, 'NaN', 'alimento'),
 (1021, 'NaN', 'alimento'),
 (1025, 'NaN', 'entretenimiento'),
 (1027, 'NaN', 'entretenimiento'),
 (1028, 'NaN', 'entretenimiento'),
 (1030, 'NaN', 'entretenimiento'),
 (1032, 'NaN', 'entretenimiento'),
 (1034, 'NaN', 'entretenimiento'),
 (1035, 'NaN', 'entretenimiento'),
 (1035, 'NaN', 'entretenimiento'),
 (1036, 'NaN', 'bazar'),
 (1040, 'NaN', 'bazar'),
 (1042, 'NaN', 'bazar'),
 (1043, 'NaN', 'bazar'),
 (1046, 'NaN', 'bazar'),
 (1052, 'NaN', 'indumentaria'),
 (1053, 'NaN', 'indumentaria'),
 (1056, 'NaN', 'indumentaria'),
 (1058, 'NaN', 'indumentaria'),
 (1059, 'NaN', 'indumentaria'),
 (1060, 'NaN', 'indumentaria'),
 (1061, 'NaN', 'indumentaria'),
 (1063, 'NaN', 'indumentaria'),
 (1065, 'NaN', 'indumentaria'),
 (1066, 'NaN', 'indumentaria'),
 (1073, 'NaN', 'perfumeria'),
 (1074, 'NaN', 'perfumeria'),
 (1075, 'NaN', 'per

In [7]:
ventasRDD.collect()

[('2019/02/28', 1016, 101.7),
 ('2019/11/24', 1052, 117.15),
 ('2019/11/02', 1028, 121.34),
 ('2019/06/29', 1058, 110.51),
 ('2019/01/04', 1063, 106.46),
 ('2019/04/30', 1081, 106.08),
 ('2019/05/26', 1028, 109.38),
 ('2019/03/10', 1056, 112.57),
 ('2019/05/01', 1063, 111.13),
 ('2019/10/11', 1075, 108.76),
 ('2019/02/28', 1032, 109.37),
 ('2019/09/30', 1099, 102.03),
 ('2019/04/25', 1004, 104.34),
 ('2019/12/29', 1019, 125.68),
 ('2019/06/07', 1079, 138.58),
 ('2019/10/10', 1035, 104.19),
 ('2019/12/03', 1036, 121.15),
 ('2019/04/16', 1007, 132.15),
 ('2019/07/06', 1027, 116.56),
 ('2019/05/08', 1060, 130.58),
 ('2019/02/15', 1061, 129.28),
 ('2019/06/10', 1079, 111.61),
 ('2019/09/28', 1011, 112.8),
 ('2019/01/10', 1092, 121.92),
 ('2019/10/09', 1021, 111.35),
 ('2019/03/03', 1034, 125.0),
 ('2019/10/17', 1042, 111.82),
 ('2019/10/01', 1065, 115.62),
 ('2019/11/09', 1098, 139.23),
 ('2019/08/22', 1074, 131.38),
 ('2019/01/10', 1042, 119.79),
 ('2019/11/30', 1025, 101.8),
 ('2019/08/0

In [8]:
# A. Cuál es el producto más vendido
cantidad_vendida = (ventasRDD.map(lambda x: (x[1],1))).reduceByKey(lambda x,y: x+1).sortBy(lambda x: -x[1])
cantidad_vendida.take(5)

[(1042, 3), (1028, 2), (1056, 2), (1040, 2), (1063, 2)]

In [9]:
# B. Cuál es la categoría de productos más vendida
# generamos el RDD con clave 'codigo' para productosRDD
productos = productosRDD.map(lambda x: (x[0],x[2]))
productos.collect()

[(1004, 'alimento'),
 (1007, 'alimento'),
 (1011, 'alimento'),
 (1016, 'alimento'),
 (1018, 'alimento'),
 (1019, 'alimento'),
 (1021, 'alimento'),
 (1025, 'entretenimiento'),
 (1027, 'entretenimiento'),
 (1028, 'entretenimiento'),
 (1030, 'entretenimiento'),
 (1032, 'entretenimiento'),
 (1034, 'entretenimiento'),
 (1035, 'entretenimiento'),
 (1035, 'entretenimiento'),
 (1036, 'bazar'),
 (1040, 'bazar'),
 (1042, 'bazar'),
 (1043, 'bazar'),
 (1046, 'bazar'),
 (1052, 'indumentaria'),
 (1053, 'indumentaria'),
 (1056, 'indumentaria'),
 (1058, 'indumentaria'),
 (1059, 'indumentaria'),
 (1060, 'indumentaria'),
 (1061, 'indumentaria'),
 (1063, 'indumentaria'),
 (1065, 'indumentaria'),
 (1066, 'indumentaria'),
 (1073, 'perfumeria'),
 (1074, 'perfumeria'),
 (1075, 'perfumeria'),
 (1079, 'perfumeria'),
 (1081, 'cuidadoCorporal'),
 (1092, 'cuidadoCorporal'),
 (1094, 'cuidadoCorporal'),
 (1098, 'cuidadoCorporal'),
 (1099, 'cuidadoCorporal')]

In [10]:
# generamos el RDD con clave 'codigo' para ventasRDD
ventas = ventasRDD.map(lambda x: (x[1],1))
ventas.collect()

[(1016, 1),
 (1052, 1),
 (1028, 1),
 (1058, 1),
 (1063, 1),
 (1081, 1),
 (1028, 1),
 (1056, 1),
 (1063, 1),
 (1075, 1),
 (1032, 1),
 (1099, 1),
 (1004, 1),
 (1019, 1),
 (1079, 1),
 (1035, 1),
 (1036, 1),
 (1007, 1),
 (1027, 1),
 (1060, 1),
 (1061, 1),
 (1079, 1),
 (1011, 1),
 (1092, 1),
 (1021, 1),
 (1034, 1),
 (1042, 1),
 (1065, 1),
 (1098, 1),
 (1074, 1),
 (1042, 1),
 (1025, 1),
 (1061, 1),
 (1046, 1),
 (1059, 1),
 (1040, 1),
 (1042, 1),
 (1066, 1),
 (1073, 1),
 (1094, 1),
 (1018, 1),
 (1056, 1),
 (1035, 1),
 (1043, 1),
 (1030, 1),
 (1035, 1),
 (1040, 1),
 (1053, 1),
 (1065, 1),
 (1081, 1)]

In [11]:
# unificamos ambos RDDs
productos_vendidos = productos.join(ventas)
productos_vendidos.collect()

[(1004, ('alimento', 1)),
 (1016, ('alimento', 1)),
 (1028, ('entretenimiento', 1)),
 (1028, ('entretenimiento', 1)),
 (1032, ('entretenimiento', 1)),
 (1036, ('bazar', 1)),
 (1040, ('bazar', 1)),
 (1040, ('bazar', 1)),
 (1052, ('indumentaria', 1)),
 (1056, ('indumentaria', 1)),
 (1056, ('indumentaria', 1)),
 (1060, ('indumentaria', 1)),
 (1092, ('cuidadoCorporal', 1)),
 (1021, ('alimento', 1)),
 (1025, ('entretenimiento', 1)),
 (1053, ('indumentaria', 1)),
 (1061, ('indumentaria', 1)),
 (1061, ('indumentaria', 1)),
 (1065, ('indumentaria', 1)),
 (1065, ('indumentaria', 1)),
 (1073, ('perfumeria', 1)),
 (1081, ('cuidadoCorporal', 1)),
 (1081, ('cuidadoCorporal', 1)),
 (1018, ('alimento', 1)),
 (1030, ('entretenimiento', 1)),
 (1034, ('entretenimiento', 1)),
 (1042, ('bazar', 1)),
 (1042, ('bazar', 1)),
 (1042, ('bazar', 1)),
 (1046, ('bazar', 1)),
 (1058, ('indumentaria', 1)),
 (1066, ('indumentaria', 1)),
 (1074, ('perfumeria', 1)),
 (1094, ('cuidadoCorporal', 1)),
 (1098, ('cuidadoCo

In [12]:
# Mapeamos a un nuevo RDD con clave 'categoria', aplicamos el reduce y ordenamos descendentemente
productos_vendidos_por_categoria = (productos_vendidos.map(lambda x: (x[1][0],x[1][1]))).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1])
productos_vendidos_por_categoria.take(1)

[('indumentaria', 14)]

In [13]:
# C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
# Generamos un RDD mapeado por clave y reducimos contabilizando cada producto
cantidad_vendida_aux = (ventasRDD.map(lambda x: (x[1],1))).reduceByKey(lambda x,y: x + y)
cantidad_vendida_aux.collect()

[(1016, 1),
 (1052, 1),
 (1028, 2),
 (1058, 1),
 (1056, 2),
 (1032, 1),
 (1004, 1),
 (1036, 1),
 (1060, 1),
 (1092, 1),
 (1034, 1),
 (1042, 3),
 (1098, 1),
 (1074, 1),
 (1046, 1),
 (1040, 2),
 (1066, 1),
 (1094, 1),
 (1018, 1),
 (1030, 1),
 (1063, 2),
 (1081, 2),
 (1075, 1),
 (1099, 1),
 (1019, 1),
 (1079, 2),
 (1035, 3),
 (1007, 1),
 (1027, 1),
 (1061, 2),
 (1011, 1),
 (1021, 1),
 (1065, 2),
 (1025, 1),
 (1059, 1),
 (1073, 1),
 (1043, 1),
 (1053, 1)]

In [14]:
# Generamos el RDD de productos con la informacion del campo 'descripcion'
productos_aux = productosRDD.map(lambda x: (x[0],x[1]))
productos_aux.take(5)

[(1004, 'NaN'), (1007, 'NaN'), (1011, 'NaN'), (1016, 'NaN'), (1018, 'NaN')]

In [15]:
# Unimos ambos RDDs con join y ordenamos. Mostramos el top5 de ventas en orden descendente
top5_ventas_aux = (cantidad_vendida_aux.join(productos_aux)).sortBy(lambda x: -x[1][0])
top5_ventas_aux.take(5)

[(1042, (3, 'NaN')),
 (1035, (3, 'NaN')),
 (1035, (3, 'NaN')),
 (1028, (2, 'NaN')),
 (1056, (2, 'NaN'))]

In [16]:
#  D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.
# Generamos los RDDs necesarios
ventas_aux2 = ventasRDD.map(lambda x: (x[1],(x[2],1)))
ventas_aux2.collect()

[(1016, (101.7, 1)),
 (1052, (117.15, 1)),
 (1028, (121.34, 1)),
 (1058, (110.51, 1)),
 (1063, (106.46, 1)),
 (1081, (106.08, 1)),
 (1028, (109.38, 1)),
 (1056, (112.57, 1)),
 (1063, (111.13, 1)),
 (1075, (108.76, 1)),
 (1032, (109.37, 1)),
 (1099, (102.03, 1)),
 (1004, (104.34, 1)),
 (1019, (125.68, 1)),
 (1079, (138.58, 1)),
 (1035, (104.19, 1)),
 (1036, (121.15, 1)),
 (1007, (132.15, 1)),
 (1027, (116.56, 1)),
 (1060, (130.58, 1)),
 (1061, (129.28, 1)),
 (1079, (111.61, 1)),
 (1011, (112.8, 1)),
 (1092, (121.92, 1)),
 (1021, (111.35, 1)),
 (1034, (125.0, 1)),
 (1042, (111.82, 1)),
 (1065, (115.62, 1)),
 (1098, (139.23, 1)),
 (1074, (131.38, 1)),
 (1042, (119.79, 1)),
 (1025, (101.8, 1)),
 (1061, (130.64, 1)),
 (1046, (123.01, 1)),
 (1059, (123.48, 1)),
 (1040, (137.61, 1)),
 (1042, (115.74, 1)),
 (1066, (133.01, 1)),
 (1073, (101.24, 1)),
 (1094, (130.48, 1)),
 (1018, (131.63, 1)),
 (1056, (110.77, 1)),
 (1035, (104.11, 1)),
 (1043, (126.91, 1)),
 (1030, (119.15, 1)),
 (1035, (139.0

In [17]:
# Agrupamos y guardamos el precio minimo y maximo alcanzado para cada producto. 
# Si el producto se vendio una unica vez, no registra cambios y en la tupla guarda un 1 en la posicion de maximo
ventas_aux3 = ventas_aux2.reduceByKey(lambda x,y: ( (x[0],y[0]) if x[0]<y[0] else (y[0],x[0])))
ventas_aux3.collect()

[(1016, (101.7, 1)),
 (1052, (117.15, 1)),
 (1028, (109.38, 121.34)),
 (1058, (110.51, 1)),
 (1056, (110.77, 112.57)),
 (1032, (109.37, 1)),
 (1004, (104.34, 1)),
 (1036, (121.15, 1)),
 (1060, (130.58, 1)),
 (1092, (121.92, 1)),
 (1034, (125.0, 1)),
 (1042, (111.82, 115.74)),
 (1098, (139.23, 1)),
 (1074, (131.38, 1)),
 (1046, (123.01, 1)),
 (1040, (137.61, 139.21)),
 (1066, (133.01, 1)),
 (1094, (130.48, 1)),
 (1018, (131.63, 1)),
 (1030, (119.15, 1)),
 (1063, (106.46, 111.13)),
 (1081, (106.08, 123.06)),
 (1075, (108.76, 1)),
 (1099, (102.03, 1)),
 (1019, (125.68, 1)),
 (1079, (111.61, 138.58)),
 (1035, (104.11, 104.19)),
 (1007, (132.15, 1)),
 (1027, (116.56, 1)),
 (1061, (129.28, 130.64)),
 (1011, (112.8, 1)),
 (1021, (111.35, 1)),
 (1065, (115.62, 127.39)),
 (1025, (101.8, 1)),
 (1059, (123.48, 1)),
 (1073, (101.24, 1)),
 (1043, (126.91, 1)),
 (1053, (123.76, 1))]

In [25]:
# Nos reservamos unicamente los productos que hayan sido vendidos al menos 2 veces
ventas_con_variacion_precio = ventas_aux3.filter(lambda x: x[1][1]!=1)
ventas_con_variacion_precio.collect()

[(1028, (109.38, 121.34)),
 (1056, (110.77, 112.57)),
 (1042, (111.82, 115.74)),
 (1040, (137.61, 139.21)),
 (1063, (106.46, 111.13)),
 (1081, (106.08, 123.06)),
 (1079, (111.61, 138.58)),
 (1035, (104.11, 104.19)),
 (1061, (129.28, 130.64)),
 (1065, (115.62, 127.39))]

In [35]:
# Mapeamos un nuevo RDD con la diferencia de precio que se registro para cada producto y mostramos el que mayor variacion sufrio
variacion_precio_productos = (ventas_con_variacion_precio.map(lambda x: (x[0],x[1][1] - x[1][0]))).sortBy(lambda x: -x[1])
variacion_precio_productos.take(1)

[(1079, 26.970000000000013)]

In [37]:
# E. Cuál es la categoria que registró mayor aumento de precio en el último año.
# Se genera un RDD union del RDD incrementos de precios con los productos con la informacion de categorias
productos_aumento_precios = variacion_precio_productos.join(productos)
productos_aumento_precios.collect()

[(1028, (11.960000000000008, 'entretenimiento')),
 (1056, (1.7999999999999972, 'indumentaria')),
 (1040, (1.5999999999999943, 'bazar')),
 (1081, (16.980000000000004, 'cuidadoCorporal')),
 (1065, (11.769999999999996, 'indumentaria')),
 (1061, (1.3599999999999852, 'indumentaria')),
 (1042, (3.9200000000000017, 'bazar')),
 (1079, (26.970000000000013, 'perfumeria')),
 (1063, (4.670000000000002, 'indumentaria')),
 (1035, (0.0799999999999983, 'entretenimiento')),
 (1035, (0.0799999999999983, 'entretenimiento'))]

In [39]:
# Se genera un nuevo RDD con clave categoria
aumento_precio_categorias_aux = productos_aumento_precios.map(lambda x: (x[1][1], x[1][0]))
aumento_precio_categorias_aux.collect()

[('entretenimiento', 11.960000000000008),
 ('indumentaria', 1.7999999999999972),
 ('bazar', 1.5999999999999943),
 ('cuidadoCorporal', 16.980000000000004),
 ('indumentaria', 11.769999999999996),
 ('indumentaria', 1.3599999999999852),
 ('bazar', 3.9200000000000017),
 ('perfumeria', 26.970000000000013),
 ('indumentaria', 4.670000000000002),
 ('entretenimiento', 0.0799999999999983),
 ('entretenimiento', 0.0799999999999983)]

In [42]:
# Se agrupan las categorias y se suman los incrementos de precios
# Se muestra la categoria que registra mayor incremento
aumento_precio_categorias = (aumento_precio_categorias_aux.reduceByKey(lambda x,y: x + y)).sortBy(lambda x: -x[1])
aumento_precio_categorias.take(1)

[('perfumeria', 26.970000000000013)]

# FINAL DEL EJERCICIO